# Surprise homework

In [1]:
import io 
import os
import pandas as pd
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold


In [2]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')
k = 30
trainset, testset = train_test_split(data, test_size=.25)

NameError: name 'train_test_split' is not defined

### A random rating based on the distribution of all ratings

In [ ]:
algo = NormalPredictor()

cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

### kNN cosine

In [ ]:
algo = KNNWithMeans(k, sim_options={'name': 'cosine'})

cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

### kNN Mean Squared Difference

In [ ]:
algo = KNNWithMeans(k, sim_options={'name': 'msd'})

cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

### kNN Pearson

In [ ]:
algo = KNNWithMeans(k, sim_options={'name': 'pearson'})

cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

### SVD

In [ ]:
algo = SVD()

cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

По средним значениям RMSE видно, что лучший алгоритм - SVD, его и будем использовать дальше

### Calculate precision@k and recall@k

In [ ]:
#from FAQ
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        
        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

algo = SVD()
kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)

#усредним по всем пользователям
precision_at_k = sum(prec for prec in precisions.values()) / len(precisions)
recall_at_k = sum(rec for rec in recalls.values()) / len(recalls)
print(precision_at_k)
print(recall_at_k)

### Predict

In [ ]:
#основано на примерах из FAQ
n = 5
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = defaultdict(list)
for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))

for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

In [ ]:
print("User 34")
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.item')
films_data = pd.read_csv(file_path, sep="|", encoding='ansi', usecols=[0,1,2], names=['id','name', 'date'])
for id, score in top_n['34']:
    name = films_data.at[int(id) - 1, 'name'] #вычитаем -1, т.к. Python
    date = films_data.at[int(id) - 1, 'date']
    print("{:<6} ({:<30} {:<11}) {:<10.3f}".format(id, name, date, score))